# Dynamic Programming

The term dynamic programming refers to a collection of algorithms that can be used to **compute optimal policies** for a Markov Decision Process (MDP), **given a perfect model of the environment**.  

Notice that these methods in some way are "cheating": **they require full access to the MDP** because they depend on **knowing the dynamics of the environment**, which is something we can’t always obtain. However, they are important theoretically. 

When an agent has full access to an MDP, there’s no uncertainty because he can look at the dynamics and rewards and calculate expectations directly. So **there’s no need for exploration, for interaction or for trial-and-error learning**. 

Suppose to have **a policy** $\pi$ (a universal plan that covers all possible states):

$\pi(a|s) = P(A_t=a | S_t=s)$

how much reward should we expect from this policy? Even though we know how to act optimally, the environment might send the agent backward to the hole. So the return is not enough and we have to look to maximize the **expected return**, in order to take into account the **environment stochasticity**. 

We need a method to find a policy, because it isn’t obvious at all what the optimal policy looks like! One immediate question that arises when looking at a policy is this: how good is this policy? How much better is this policy compared to another policy?

Consider the following policies for the frozen-lake environment:

<img src="./images/frozen_lake_policies_comparison.png" width="600">

For example, we can use a brute-force approach: we can simulate the environment for a long time and then average the returns and evaluate the success probability:

In [4]:
import numpy as np

def evaluate(env, pi, goal_state, n_episodes=100, max_steps=200):
    success = 0;
    results = []
    for _ in range(n_episodes):
        done = False;
        steps = 0;
        state, _ = env.reset();
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _, _ = env.step(pi(state))
            results[-1] += reward;
            steps += 1
        if(state == goal_state):
            success += 1;
    return (success/n_episodes)*100, np.mean(results);

 We ca use Gymnasium to run the algorithm. Notice that  it generally wrap the environment using a single class, called Env. This class exposes the common most essential methods of any environment (like step and reset). Having this interface class is great, because it allows our code to be environment agnostic. However, if we want to access the behind the scenes dynamics of a specific environment, then we cane use the "unwrapped" property.

In [5]:
import gymnasium as gym

frozen_lake = gym.make('FrozenLake-v1')
P = frozen_lake.env.unwrapped.P
goal_state = 15

LEFT, DOWN, RIGHT, UP = range(4)

In [6]:
go_get_pi = lambda s: {
    0:RIGHT, 1:RIGHT, 2:DOWN, 3:LEFT,
    4:DOWN, 5:LEFT, 6:DOWN, 7:LEFT,
    8:RIGHT, 9:RIGHT, 10:DOWN, 11:LEFT,
    12:LEFT, 13:RIGHT, 14:RIGHT, 15:LEFT
}[s]

In [7]:
probability_success, mean_return = evaluate(frozen_lake, go_get_pi, goal_state=goal_state);

print("Reaches goal ", probability_success, "%");
print("Obtains an average undiscounted return of ", mean_return);

Reaches goal  4.0 %
Obtains an average undiscounted return of  0.04


In [8]:
careful_pi = lambda s: {
    0:LEFT, 1:UP, 2:UP, 3:UP,
    4:LEFT, 5:LEFT, 6:UP, 7:LEFT,
    8:UP, 9:DOWN, 10:LEFT, 11:LEFT,
    12:LEFT, 13:RIGHT, 14:RIGHT, 15:LEFT
}[s]

In [6]:
probability_success, mean_return = evaluate(frozen_lake, careful_pi, goal_state=goal_state);

print("Reaches goal ", probability_success, "%");
print("Obtains an average undiscounted return of ", mean_return);

Reaches goal  55.00000000000001 %
Obtains an average undiscounted return of  0.55


It seems being a go-get-it policy doesn’t pay well in the FL environment! However, the brute-force approach is not a good idea, because it is very inefficient. We need a better way to evaluate policies.

## State-value function

We can put numbers to states for a given policy. That is, if we’re given a policy, we can calculate the expected return (remember, we are under stochastic environments, so we must account for all the possible ways the environment can react to our policy, so the expectation) starting from every single state. This is called the **state-value function (or V-function)** denoted $v_\pi(s)$:

$\displaystyle v_{\pi}(s)=E_{\pi}(G_t|S_t=s)=E_{\pi}\left(\sum\limits_{k=0}^{\infty }{\gamma^kR_{t+k+1}}|S_t=s \right) $

For example, if we follow the "go-get-it" policy starting from state 14, we get the following value:

<img src="./images/frozen_lake_state_value_function.png" width="600">

Okay, it isn’t straightforward to calculate because of the dependence on the values of other states (10 and 14, in this case), which we don’t have either and one is the same state!

We can use the recursive definition of the return:

$\displaystyle v_{\pi}(s)=E_{\pi}(R_{t+1}+\gamma G_{t+1}|S_t=s)= $

applying the law of [total expectation](https://en.wikipedia.org/wiki/Law_of_total_expectation):

$\displaystyle =\sum\limits_{a}{\pi(a|s)} \left(E_{\pi}(R_{t+1}+\gamma G_{t+1}|S_t=s, A_t=a\right)= $

then splitting the expectation of a sum into a sum of expectations and note that, given an action, the expected immediate reward doesn't depend on the policy:

$\displaystyle =\sum\limits_{a}{\pi(a|s)} \left(E(R_{t+1}|S_t=s, A_t=a)+\gamma E_{\pi}(G_{t+1}|S_t=s, A_t=a)\right)= $

we can write the expected immediate reward in terms of the system dynamics:

$\displaystyle =\sum\limits_{a}{\pi(a|s)}\left(\sum\limits_{r}{r\sum\limits_{s'}{p(s',r|s,a)}}+\gamma E_{\pi}(G_{t+1}|S_t=s, A_t=a)\right)= $

applying the law of total expectation on the next state:

$\displaystyle =\sum\limits_{a}{\pi(a|s)}\left( \sum\limits_{s',r}{p(s',r|s,a)}r+\gamma \sum\limits_{s',r}{p(s',r|s,a)E_{\pi}(G_{t+1}|S_t=s, A_t=a, S_{t+1}=s')} \right) = $

by the Markov property, knowing $S_{t+1}$ makes the expectation independent of $S_t$ and $A_t$:

$\displaystyle = \sum\limits_{a}{\pi(a|s)}\left( \sum\limits_{s',r}{p(s',r|s,a)}r+\gamma \sum\limits_{s',r}{p(s',r|s,a)E_{\pi}(G_{t+1}|S_{t+1}=s')} \right) = $

acknowledging that 

$\displaystyle E_{\pi}(G_{t+1}|S_{t+1}=s')=v_{\pi}(s')$ 

and combining summations we obtain **the Bellman equation**:

$\displaystyle v_{\pi}(s)=\sum\limits_{a}{\pi(a|s)} \sum\limits_{s',r}{p(s',r|s,a)}(r+\gamma v_{\pi}(s')) $

The value of a state must equal the (discounted) value of the expected next state, plus the reward expected along the way:

<img src="./images/bellman_equation.png" width="600">

Notice how the value of a state depends recursively on the value of possibly many other states, which values may also depend on others, including the original state!

## Action-value function

However, the critical question is not merely about the value of a state, but the value of taking an action in a state. For instance, notice that the "Go-get-it" policy goes right when in state 14, but the "Careful" policy goes down, but which action is better? More specifically, which action is better under each policy?  

The **action-value function (or Q-function)** captures precisely this: the expected return if the agent follows a policy $\pi$ after taking action $a$ in state $s$:

$\displaystyle q_{\pi}(s,a)=E_{\pi}(G_t|S_t=s, A_t=a)=E_{\pi}\left(\sum\limits_{k=0}^{\infty }{\gamma^kR_{t+k+1}}|S_t=s, A_t=a \right) $


With can provide a formulation of the Bellman equation also for the action-value function with similar reasoning:

$\displaystyle q_{\pi}(s,a)=\sum\limits_{s',r}{p(s',r|s,a)}(r+\gamma \sum\limits_{a'}\pi(a'|s')q_{\pi}(s',a') $

These value functions can be **estimated from experience**. 

For example, if an agent follows a policy and maintains an average, for each state encountered, of the actual returns that have followed that state, then the average will converge to the state value as the number of times that state is encountered approaches infinity. If separate averages are kept for each action taken in each state, then these averages will similarly converge to the action values. Of course, if there are many states, then it may not be practical to keep separate averages for each state or action individually, instead, the agent can maintain parameterized esteems (with fewer parameters than states) and adjust the parameters to better match the observed returns. 

## Optimal Policies

Policies, state-value functions and action-value functions are the components we use to describe, evaluate, and improve behaviors. We call it **optimality** when these components are the best they can be. 

Solving a reinforcement learning task means, roughly, finding a policy that achieves a lot of reward over the long run. Value functions define an ordering over policies. A policy $\pi$ is defined to be better than or equal to a policy $\pi'$ if its expected return is greater than or equal to that of $\pi'$ for all states:

$\displaystyle \pi >= \pi' \text{ if and only if } v_{\pi}>=v_{\pi'} \forall s \in S$

There is always at least one policy that is better than or equal to all other
policies. This is an **optimal policy** $\pi*$.

Similarly, an **optimal state-value function** is a state-value function with the maximum value across all policies for all states:

$\displaystyle v_*(s)=\underset{\pi }{\text{max }}v_{\pi}(s)\qquad \forall \in S$

Finally, an **optimal action-value function** is an action-value function with the maximum value across all policies for all state-action pairs. 

$\displaystyle q_*(s,a)=\underset{\pi }{\text{max }}q_{\pi}(s,a)\qquad \forall \in S \text{ and } \forall a \in A$

Notice that if we had the optimal state-value function, we could use the MDP to do a one-step search for the optimal action-value function:

$\displaystyle q_*(s,a)= E(R_{t+1} + \gamma v_*(S_{t+1}) | S_t=s, A_t=a)$

If we had the optimal action-value function, we don’t need the MDP at all. We can use it to find the optimal state-value function by merely taking the maximum over the actions:

$\displaystyle v_*(s)=\underset{a}{\text{max }}q_*(s,a) $

Then, we can write:

$\displaystyle v_*(s)=\underset{a}{\text{max }}E(R_{t+1}+\gamma v_*(S_{t+1})|S_t=s, A_t=a)$

$\displaystyle = \underset{a}{\text{max }} \sum\limits_{s',r}{p(s',r|s,a)}[r+\gamma v_*(s')] $

This means that we take the max action of the weighted sum of the reward and discounted optimal value of the next state.

Similarly, the optimal action-value function can be obtained this way:

$\displaystyle q_*(s,a)= \sum\limits_{s',r}{p(s',r|s,a)}[r+\gamma \underset{a'}{\text{max }} q_*(s',a')] $

These equations are called the **Bellman optimality equations** and they are a system of equations and if the dynamic of the environment is known, then in principle we can solve it. Once one has one of them, it is easy to determine an optimal policy. 

This solution relies on at least three assumptions that are rarely true in practice: 

- we accurately know the dynamics of the environment;  
- we have enough computational resources to complete the computation of the solution; 
- the Markov property.  

Generally various combinations of these assumptions are violated and in reinforcement learning typically we have to settle for approximate solutions.

## Policy Evaluation

We need an algorithm for evaluating an arbitrary policy. Such an algorithm is known as an **policy evaluation** and consists of calculating the state-value function for a given policy by sweeping through the state space and iteratively improving estimates. The initial approximation is chosen arbitrarily and each successive approximation is obtained by using the Bellman equation as an **update rule**:

$\displaystyle v_{k+1}(s)=E_{\pi}(R_{t+1} + \gamma v_k(S_{t+1}|S_t=s))=\sum\limits_{a}{\pi(a|s)}( \sum\limits_{s',r}{p(s',r|s,a)}(r+\gamma v_{k}(s'))) $

this sequence can be shown to converge to the value function as $k$ goes to infinity, under the same conditions that guarantee the existence of the value function. To produce each successive approximation, we apply the same operation to each state: we replace the old value of state $s$ with a new value obtained from the old values of the successor of $s$, and the expected immediate rewards, along all the one-step transitions possible under the policy being evaluated.

Formally, iterative policy evaluation converges only in the limit, but in practice it must be halted short of this, for example we can tests the quantity:

$\displaystyle \underset{s \in S}{\text{max }}|v_{k+1}(s)−v_k(s)|$ 

and stops when it is sufficiently small.

The same update rule can be alo applied to the action-value function:

$\displaystyle q_{k+1}(s,a)=\sum\limits_{s',r}{p(s',r|s,a)}(r+\gamma \sum\limits_{a'}\pi(a'|s')q_{k}(s',a') $

Note that update estimates of the values of states are based on estimates of the values of successor states. That is, they update estimates on the basis of other estimates. We call this general idea **bootstrapping**.

Also, it’s important to notice that the k here are **iterations across estimates**, but they’re **not interactions with the environment**. These aren’t episodes, and the agent isn't selecting actions and observing the environment. These aren’t time steps either. Instead, these are the iterations of the iterative policy evaluation algorithm.

We can write the policy evaluation algorithms in Python:

In [9]:
# We need the policy to evaluate and the MDP the policy runs on 
# The discount factor and gamma are defaults to 1
# Theta is a small number that we use to check for convergence

def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    
    # initialize the first-iteration estimates to zero.
    prev_V = np.zeros(len(P))
    
    # looping forever...
    while True:
        # initialize the current-iteration estimates to zero as well.
        V = np.zeros(len(P))
        
        # loop through all states to estimate the state-value function
        for s in range(len(P)):
            
            # we use the policy pi to get the possible transitions,
            # each transition tuple has a probability, next state, 
            # reward, and a done flag indicating whether the next_state 
            # is terminal or not
            for prob, next_state, reward, done in P[s][pi(s)]:
                
                # calculate the value of that state by summing up the 
                # weighted value of that transition
                V[s] += prob * (reward + gamma * prev_V[next_state])
        
        # at the end of each iteration (a state sweep), we make sure 
        # that the state-value functions are changing;
        # otherwise, we call it converged
        if np.max(np.abs(prev_V - V)) < theta:
            break
        
        # finally, copy to get ready for the next iteration or 
        prev_V = V.copy()
        
    # return the latest state-value function    
    return V

We can write also a simple function to print the value function in a readable way:

In [10]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

Let’s now run policy evaluation for the two policies "Go-get-it" and "Careful" for the FL environment.

In [11]:
V = policy_evaluation(go_get_pi, P, gamma=0.99)
print_state_value_function(V, P, prec=4)

State-value function:
| 00 0.0342 | 01 0.0231 | 02 0.0468 | 03 0.0231 |
| 04 0.0463 |           | 06 0.0957 |           |
| 08  0.094 | 09 0.2386 | 10 0.2901 |           |
|           | 13 0.4329 | 14 0.6404 |           |


In [13]:
V = policy_evaluation(careful_pi, P, gamma=0.99)
print_state_value_function(V, P, prec=4)

State-value function:
| 00 0.4079 | 01 0.3754 | 02 0.3543 | 03 0.3438 |
| 04 0.4203 |           | 06 0.1169 |           |
| 08 0.4454 | 09  0.484 | 10 0.4328 |           |
|           | 13 0.5884 | 14 0.7107 |           |


<img src="./images/frozen_lake_policies_evaluation.png" width="600">

As already seen, the Go-get-it policy doesn’t pay well in the FL environment! Whereas the Careful policy is much better. Fascinating results, but a question arises: are there any better policies for this environment?

## Policy Improvement

The reason for computing the value function for a policy is to help find better policies. Suppose we have determined the value function $v_{\pi}$ for a policy $\pi$. For a state $s$ we would like to know if we should change the policy and choose an action $a \neq \pi(s)$. We know how good is to follow the current policy (it is $v_{\pi}(s)$), but would it be better to change? One way to answer this question is to consider selecting $a$ in $s$ and thereafter following the existing policy $\pi$:

$\displaystyle q_{\pi}(s,a)=\sum\limits_{s',r}{p(s',r|s,a)}(r+\gamma v_{\pi}(s')) $

The key criterion is whether this is greater than or less than $v_{\pi}(s)$. If it is greater, then it is better to select $a$ in $s$ than follow $\pi$. This is a case of the general **policy improvement theorem**. Let $\pi$ and $\pi'$ a pair of policies that:

$\displaystyle q_{\pi}(s,\pi'(s)) \geq v_{\pi}(s) \enspace \forall s \in S \Rightarrow v_{\pi'}(s) \geq v_{\pi}(s) $

This result applies to the two policies that we considered before. The original policy $\pi$ and a changed policy that is identical to the first, except for the different action taken in one state $s$. Thus, if $q_{\pi}(s, a) > v_{\pi}(s)$, then the changed policy is indeed better than the original one.

It is natural to extend this and to consider changes at all states and to all possible actions, selecting at each state the action that appears best according to $q_{\pi}(s, a)$:

$\displaystyle \pi'(s) = \underset{a}{\arg \max}\> q_{\pi}(s,a) = \underset{a}{\arg \max}\> \sum\limits_{s',r}{p(s',r|s,a)}(r+\gamma v_{\pi}(s') $

The process of making a new policy that improves an original policy, by making it greedy with respect to the value function of the original policy, is called **policy improvement**. Let's try to improve the Careful policy:

<img src="./images/frozen_lake_careful_policy.png" width="600">

Notice how if we act greedily with respect to the  value function of the policy, we obtain a new policy: careful-plus.

Let's implement it in Python:

In [48]:
# It takes the state-value function of the policy
# we want to improve, V, and the MDP, P (and gamma, optionally)
def policy_improvement(V, P, gamma=1.0):
    
    # initialize the Q-function to zero (technically, we
    # can initialize these randomly, but let’s keep things simple)
    Q = np.zeros((len(P), len(P[0])))
    
    # loop through the states, actions, and transitions
    # and calculate the action-value function
    for s in range(len(P)):
        for a in range(len(P[s])):
            for prob, next_state, reward, done in P[s][a]:
                Q[s][a] += prob * (reward + gamma * V[next_state])
                
    # obtain a new, greedy policy by taking the argmax of the Q-function            
    new_pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
    
    return new_pi

We can create a support function in order to visualize policies:

In [49]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='policy'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

Then we can try to improve the careful policy:

In [50]:
careful_plus_pi = policy_improvement(V, P, gamma=0.99)

In [51]:
print_policy(careful_plus_pi, P)

policy
| 00      < | 01      ^ | 02      > | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


Is this policy any better? Well, we can evaluate it with brute-force or using the policy evaluation algorithm:

In [52]:
probability_success, mean_return = evaluate(frozen_lake, careful_plus_pi, goal_state=goal_state);

print("Reaches goal ", probability_success, "%");
print("Obtains an average undiscounted return of ", mean_return);

Reaches goal  82.0 %
Obtains an average undiscounted return of  0.82


In [21]:
V = policy_evaluation(careful_plus_pi, P, gamma=0.99)
print_state_value_function(V, P, prec=4)

State-value function:
| 00  0.542 | 01 0.4988 | 02 0.4707 | 03 0.4569 |
| 04 0.5585 |           | 06 0.3583 |           |
| 08 0.5918 | 09 0.6431 | 10 0.6152 |           |
|           | 13 0.7417 | 14 0.8628 |           |


<img src="./images/frozen_lake_careful_plus_policy.png" width="600">

The new policy is better than the original policy. This is great, acting greedily with respect to the action-value function gave us an improved policy. This is what the policy-improvement algorithm does: **it calculates an action-value function using the state value function and the MDP, and it returns a greedy policy with respect to the action-value function of the original policy**.

The natural next questions are these: is there a better policy than this one? Can we do any better than careful-plus? Can we evaluate the careful-plus policy, and then improve it again? Maybe! But, there’s only one way to find out. Let’s give it a try!

In [33]:
V = policy_evaluation(careful_plus_pi, P, gamma=0.99)
careful_plus_plus_pi = policy_improvement(V, P, gamma=0.99)

In [34]:

print_policy(careful_plus_plus_pi, P)

policy
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [35]:
probability_success, mean_return = evaluate(frozen_lake, careful_plus_plus_pi, goal_state=goal_state);

print("Reaches goal ", probability_success, "%");
print("Obtains an average undiscounted return of ", mean_return);

Reaches goal  75.0 %
Obtains an average undiscounted return of  0.75


There’s no improvement this time. No improvement occurs because the careful-plus policy is an optimal policy of the FL environment. We only needed one improvement over the careful policy because this policy was good to begin with.

Even if we start with an **adversarial policy** designed to perform poorly, **alternating over policy evaluation and improvement** would still end up with an optimal policy:

In [36]:
adversarial_pi = lambda s: {
    0:UP, 1:UP, 2:UP, 3:UP,
    4:UP, 5:LEFT, 6:UP, 7:LEFT,
    8:LEFT, 9:LEFT, 10:LEFT, 11:LEFT,
    12:LEFT, 13:LEFT, 14:LEFT, 15:LEFT
}[s]

In [37]:
print_policy(adversarial_pi, P)

policy
| 00      ^ | 01      ^ | 02      ^ | 03      ^ |
| 04      ^ |           | 06      ^ |           |
| 08      < | 09      < | 10      < |           |
|           | 13      < | 14      < |           |


In [38]:
probability_success, mean_return = evaluate(frozen_lake, adversarial_pi, goal_state=goal_state);

print("Reaches goal ", probability_success, "%");
print("Obtains an average undiscounted return of ", mean_return);

Reaches goal  0.0 %
Obtains an average undiscounted return of  0.0


In [45]:
V = policy_evaluation(adversarial_pi, P, gamma=0.99)
adversarial_pi_2 = policy_improvement(V, P, gamma=0.99)

V = policy_evaluation(adversarial_pi_2, P, gamma=0.99)
adversarial_pi_3 = policy_improvement(V, P, gamma=0.99)

V = policy_evaluation(adversarial_pi_3, P, gamma=0.99)
adversarial_pi_4 = policy_improvement(V, P, gamma=0.99)

V = policy_evaluation(adversarial_pi_4, P, gamma=0.99)
adversarial_pi_5 = policy_improvement(V, P, gamma=0.99)

In [46]:
print_policy(adversarial_pi_5, P)

policy
| 00      < | 01      ^ | 02      > | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [47]:
probability_success, mean_return = evaluate(frozen_lake, adversarial_pi_5, goal_state=goal_state);

print("Reaches goal ", probability_success, "%");
print("Obtains an average undiscounted return of ", mean_return);

Reaches goal  74.0 %
Obtains an average undiscounted return of  0.74


## Policy Iteration

Once a policy has been improved using its value function to yield a better policy, we can then compute a new value function and improve again to yield an even better policy. We can thus obtain a sequence of monotonically improving policies and value functions:

$\pi_0 \overset{evaluate}{\longrightarrow }  v_{\pi_0} \overset{improve}{\longrightarrow } \pi_1 \overset{evaluate}{\longrightarrow }  v_{\pi_1} \overset{improve}{\longrightarrow } \pi_2 \overset{evaluate}{\longrightarrow } ... \overset{improve}{\longrightarrow } \pi_* \overset{evaluate}{\longrightarrow }  v_{\pi_*}$ 

Each policy is guaranteed to be a strict improvement over the previous one (unless it is already optimal). Because a finite MDP has only a finite number of policies, this process must converge to an optimal policy in a finite number of iterations. This way of finding an optimal policy is called **policy iteration**.

We can put all toghether in Python:

In [61]:
# it only needs the MDP (including gamma)
def policy_iteration(P, gamma=1.0, theta=1e-10):
    
    # create a random policy: a list of random actions for each state
    random_actions = np.random.choice(tuple(P[0].keys()), len(P))
    pi = lambda s: {s:a for s, a in enumerate(random_actions)}[s]
    
    while True:
        # keep a copy of the policy before modify it
        old_pi = {s:pi(s) for s in range(len(P))}
        
        # get the state-value function of the policy
        V = policy_evaluation(pi, P, gamma, theta)
        
        # get an improved policy
        pi = policy_improvement(V, P, gamma)
        
        # if it’s different, we do it all over again
        if old_pi == {s:pi(s) for s in range(len(P))}:
            break
    
    # if it’s not, we break out of the loop and return 
    # the optimal policy and the optimal state-value function 
    return V, pi

Let’s try it:

In [54]:
V_best_p, pi_best_p = policy_iteration(P, gamma=0.99)

In [55]:
print_policy(pi_best_p, P)

policy
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [56]:
print_state_value_function(V_best_p, P, prec=4)

State-value function:
| 00  0.542 | 01 0.4988 | 02 0.4707 | 03 0.4569 |
| 04 0.5585 |           | 06 0.3583 |           |
| 08 0.5918 | 09 0.6431 | 10 0.6152 |           |
|           | 13 0.7417 | 14 0.8628 |           |


In [57]:
probability_success, mean_return = evaluate(frozen_lake, pi_best_p, goal_state=goal_state);

print("Reaches goal ", probability_success, "%");
print("Obtains an average undiscounted return of ", mean_return);

Reaches goal  86.0 %
Obtains an average undiscounted return of  0.86


As mentioned, alternating policy evaluating and policy improvement yields an optimal policy and state-value function regardless of the policy you start with.  Notice how we use "an" optimal policy, but also use "the" optimal state-value function. This is not a coincidence, this is a property: an MDP can have more than one optimal policy, but it can only have a single optimal state-value function. A state-value function may have actions that are equally valued for a given state, in this case, there could be multiple optimal policies, each optimal policy selecting a different, but equally valued, action. Take a look at the FL example:

<img src="./images/frozen_lake_optimal_policies.png" width="600">

So far we have considered the special case of deterministic policies, anyway all the ideas extend easily to stochastic policies. In particular, if there are several actions at which the maximum is achieved, in the stochastic case we need not select a single action from among them. Instead, each maximizing action can be given a portion of the probability of being selected in the new greedy policy.

## Value Iteration

Policy interaction works, but slowly: each of its iterations involves policy evaluation, which is a protracted iterative computation requiring multiple sweeps through the state set. However, if we truncated policy evaluation after a single iteration, we could still improve upon the initial policy by taking the greedy policy of the Q-function estimation after a single state-space sweep of policy evaluation. This algorithm is another fundamental algorithm in RL: it’s called **value iteration**.  It can be thought of "greedily greedifying policies", because we calculate the greedy policy as soon as we can, greedily. Value iteration **doesn’t wait until we have an accurate estimate of the policy before it improves it, but instead, it truncates the policy-evaluation phase after a single state sweep**. 

We can merge a truncated policy-evaluation step and a policy improvement into the same equation. Instead of improving the policy (by taking the argmax to get a better policy and then evaluating this improved policy to obtain a value function again), we directly calculate the maximum (max, instead of argmax) value across the actions to be used for the next sweep over the states:

$\displaystyle
v_{k+1}(s) = \underset{a}{\max}\> \sum\limits_{s',r}{p(s',r|s,a)}(r+\gamma v_{\pi}(s')) $

For arbitrary $v_0$, the sequence ${v_k}$ can be shown to converge to $v_*$ under the same conditions that guarantee the existence of $v_*$. Like policy evaluation, value iteration formally requires an infinite number of iterations to converge exactly to $v_*$. In practice, we stop once the value function changes by only a small amount in a sweep.

Notice that in practice, in value iteration, we don’t have to deal with policies at all. It doesn’t have any separate evaluation phase that runs to convergence. While the goal is the same as of policy iteration (to find the optimal policy) it happens to do this through the value functions (thus the name). Only at the end of the algorithm, after the value function converges to the optimal, do we extract the optimal policy by taking the argmax over the actions of the Q-function, as before. 

We can implement this algorithm in Python: 

In [62]:
def value_iteration(P, gamma=1.0, theta=1e-10):
    
    # initialize a state-value function
    V = np.zeros(len(P))
    
    while True:
        Q = np.zeros((len(P), len(P[0])))
        
        # for every transition of every action in every state, we...
        for s in range(len(P)):
            for a in range(len(P[s])):
                for prob, next_state, reward, done in P[s][a]:    
                    # ...calculate the action-value function
                    Q[s][a] += prob * (reward + gamma * V[next_state])
        
        # After each sweep over the state space, make sure the state-value function keeps changing. 
        # Otherwise, we found the optimal V-function and should break out
        if np.max(np.abs(V - np.max(Q, axis=1))) < theta:
            break
            
        # we don’t need a separate policy-improvement phase
        V = np.max(Q, axis=1)
    
    # only at the end, we extract the optimal policy
    pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
    
    return V, pi

And again, we can solve the Frozen Lake problem:

In [63]:
V_best, pi_best = value_iteration(P, gamma=0.99)

In [64]:
print_policy(pi_best, P)

policy
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [65]:
print_state_value_function(V_best, P, prec=4)

State-value function:
| 00  0.542 | 01 0.4988 | 02 0.4707 | 03 0.4569 |
| 04 0.5585 |           | 06 0.3583 |           |
| 08 0.5918 | 09 0.6431 | 10 0.6152 |           |
|           | 13 0.7417 | 14 0.8628 |           |


In [66]:
probability_success, mean_return = evaluate(frozen_lake, pi_best, goal_state=goal_state);

print("Reaches goal ", probability_success, "%");
print("Obtains an average undiscounted return of ", mean_return);

Reaches goal  80.0 %
Obtains an average undiscounted return of  0.8


Let solve the problem also for the no discounting case ($\gamma = 1$) and for a different discounting value ($\gamma = 0.95$):

In [70]:
V_best, pi_best = value_iteration(P, gamma=1.00)
print('Optimal state-value function:')
print_state_value_function(V_best, P)

Optimal state-value function:
State-value function:
| 00  0.824 | 01  0.824 | 02  0.824 | 03  0.824 |
| 04  0.824 |           | 06  0.529 |           |
| 08  0.824 | 09  0.824 | 10  0.765 |           |
|           | 13  0.882 | 14  0.941 |           |


In [71]:
V_best, pi_best = value_iteration(P, gamma=0.95)
print('Optimal state-value function:')
print_state_value_function(V_best, P)

Optimal state-value function:
State-value function:
| 00   0.18 | 01  0.155 | 02  0.153 | 03  0.133 |
| 04  0.209 |           | 06  0.176 |           |
| 08   0.27 | 09  0.375 | 10  0.404 |           |
|           | 13  0.509 | 14  0.724 |           |


We find that they are not the same. Introducing discounting in a stochastic environment can either have no effect or **favor more risky but faster (less discounting) policies**.

A major drawback of the dynamic programming methods is that they require a perfect model of the environment, these methods in a way **"cheat" because they require full access to the MDP**, which is something we can’t always obtain. 

Another problem is that they **involve operations over the entire state set of the MDP**. If the state set is very large, then even a single sweep can be prohibitively expensive. For example, the game of backgammon has over $10^{20}$ states. **Asynchronous dynamic programming** are iterative algorithms that are not organized in terms of systematic sweeps of the state set, but they update values of states in any order. To converge correctly, however, they must continue to update the values of all the states. Of course, avoiding sweeps does not necessarily mean that we can get away with less computation. It just means that an algorithm does not need to get locked into any hopelessly long sweep before it can make progress improving a policy.

## Generalization

The dynamic programming approach consists of **two simultaneous, interacting processes, one making the value function consistent with the current policy (evaluation), and the other making the policy greedy with respect to the current value function (improvement)**. These two processes can alternate, each completing before the other begins (policy iteration); or only a single iteration of evaluation is performed before each policy improvement; moreover, the evaluation and improvement can be interleaved at an even finer grain (asynchronous methods). As long as both processes continue to update all states, the ultimate result is typically the same: convergence to the optimal value function and an optimal policy.

We use the term **generalized policy iteration** to refer to the **general idea of letting evaluation and improvement processes interact**, independently from the granularity and other details of the two processes. 

Almost all reinforcement learning methods are well described as generalized policy iteration approached: all have policies and value functions, with the policy always being improved with respect to the value function and the value function always being driven toward the policy. If both the processes stabilize (no longer produce changes), then the value function and policy must be optimal. 

The evaluation and improvement processes can be viewed as both **competing and cooperating**. They compete in the sense that they pull in opposing directions. Making the policy greedy with respect to the value function typically makes the value function incorrect for the changed policy, and making the value function consistent with the policy typically causes that policy no longer to be greedy. In the long run, however, these two processes interact to find a single joint solution: the optimal value function and an optimal policy.

<img src="./images/generalization.png" width="600">